## Подготовка ноутбука
Импортируем все необходимые библиотеки, настроим флаги и константы, которыми будем пользоваться в эксперименте, и определим служебные функции, чтобы не отвлекаться на них в расчетной части эксперимента.

In [1]:
# Если что-то меняется в коде, мы хотим это сразу видеть здесь
%load_ext autoreload
%autoreload 2

In [9]:
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

from tabulate import tabulate
from tqdm.notebook import tqdm

from itertools import product
from collections import namedtuple
from copy import deepcopy
import time
import numpy as np
import pandas as pd

# Подключаем локальные пакеты
import pyons
from pyons.models.rfid import pyradise
from pyons.models.rfid.serialization import Input

In [4]:
# Настраиваем matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

Определим переменные окружения, которые будут использоваться в эксперименте.

In [5]:
# Нужно ли пересчитывать все, или можно использовать результаты из файлов
FORCE_SIMULATION = False
SIM_FILE_PATH = 'rfid_sim_data.csv'

# Зададим число машин, проезд которых по сети мы будем моделировать.
# Чем выше это число, тем точнее результаты, но на их получение нужно больше времени.
NUM_PACKETS = 10

# Цветовая схема для графиков
CMAP_NAME = 'viridis'

In [6]:
def get_color(x):
    """
    Получить цвет из текущей карты.
    """
    return cm.get_cmap(CMAP_NAME)(x)

## Создание датасета

In [16]:
BASE_INPUT = Input(tari="12.5", m="4", trext=False, q=4, reader_tx_power="31.5", vehicle_speed=60)
SPEED = [30, 60, 90, 120]

def make_input(**kwargs) -> Input:
    inp = deepcopy(BASE_INPUT)
    for key, value in kwargs.items():
        if key not in Input.__dataclass_fields__:
            raise KeyError(f'Input does not have key "{key}"')
        setattr(inp, key, value)
    return inp

In [17]:
inputs = []
inputs.extend([make_input(tari=tari, vehicle_speed=speed) for tari, speed in product(["6.25", "12.5", "18.75", "25"], SPEED)])
inputs.extend([make_input(m=m, vehicle_speed=speed) for m, speed in product([1, 2, 4, 8], SPEED)])

In [21]:
df = pd.DataFrame(inputs)
df.to_csv(SIM_FILE_PATH, index=False, )